### Get Data

Scrape data from the following wikipedia page: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import pandas as pd
import numpy as np

url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url)
headings = ['Postal Code', 'Borough', 'Neighborhood']
tables

[    Postal Code           Borough  \
 0           M1A      Not assigned   
 1           M2A      Not assigned   
 2           M3A        North York   
 3           M4A        North York   
 4           M5A  Downtown Toronto   
 ..          ...               ...   
 175         M5Z      Not assigned   
 176         M6Z      Not assigned   
 177         M7Z      Not assigned   
 178         M8Z         Etobicoke   
 179         M9Z      Not assigned   
 
                                           Neighborhood  
 0                                         Not assigned  
 1                                         Not assigned  
 2                                            Parkwoods  
 3                                     Victoria Village  
 4                            Regent Park, Harbourfront  
 ..                                                 ...  
 175                                       Not assigned  
 176                                       Not assigned  
 177                

In [2]:
df_nb = tables[0]
df_nb

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Clean data frame

In [3]:
# remove cells with a borough that is "Not assigned"
df_nb.drop(df_nb[df_nb.Borough =='Not assigned'].index, inplace = True)
df_nb.reset_index(drop=True, inplace = True)
df_nb

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
# check if there are any rows with Neighborhood value 'Not assigned'
df_nb['Neighborhood'].str.contains('Not Assigned').value_counts()

False    103
Name: Neighborhood, dtype: int64

### Obtain the latitude and longitude for each location

In [5]:
file_path = 'https://cocl.us/Geospatial_data'
geo_data = pd.read_csv(file_path)
geo_data

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [6]:
# add lat, long to dataframe
toronto_data = df_nb.merge(geo_data,on='Postal Code')
toronto_data

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [7]:
# check if there are any NaN values
toronto_data.isnull().values.any()

False

### Create a map of Toronto with the neighborhoods superimposed.

In [8]:
# latitude, longitude of Toronto
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="TO_explorer")
location = geolocator.geocode("Toronto ON")
lat_TO = location.latitude
long_TO = location.longitude
print('The geographical coordinates for Toronto are {}, {}.'.format(lat_TO,long_TO))

The geographical coordinates for Toronto are 43.6534817, -79.3839347.


In [9]:
import folium

# create map of toronto using lat and long values
map_TO = folium.Map(location=[lat_TO,long_TO], zoom_start = 10)
map_TO

In [10]:
# add neighborhood markers to map

for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_TO)  
    
map_TO

### Explore North York Borough

In [11]:
# create new dataframe containing data for only the North York borough
ny_data = toronto_data[toronto_data['Borough'] == 'North York'].reset_index(drop=True)
ny_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [12]:
# Get the geospatial coordinates for north york
location = geolocator.geocode("North York ON")
lat_ny = location.latitude
long_ny = location.longitude
print('The geographical coordinates for North York are {}, {}.'.format(lat_ny,long_ny))

The geographical coordinates for North York are 43.7543263, -79.44911696639593.


In [14]:
# create map of North York using latitude and longitude values
map_ny = folium.Map(location=[lat_ny, long_ny], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ny_data['Latitude'], ny_data['Longitude'], ny_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

Use the Foursquare API to explore the neighborhoods in North York.

In [15]:
CLIENT_ID = 'foursquare ID' # insert your Foursquare ID
CLIENT_SECRET = 'foursquare secret' # insert your Foursquare Secret
VERSION = '20200620' # Foursquare API version

Explore the first neighborhood on the list: Parkwoods

In [17]:
# get top 10 venus that are within 500 meters radius of Parkwoods
LIMIT = 10 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    ny_data.Latitude[0], 
    ny_data.Longitude[0], 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=IFOVJ1OA1R0RPKQTKKE3UZOPGCJROGT2NTOMCFAVL5TZHM0L&client_secret=N0AUBQIUXRLR2TLNCOCGT2JMFVWEHHN0KDXB0IP34S2OO1YB&v=20200620&ll=43.7532586,-79.3296565&radius=500&limit=10'

In [18]:
import requests

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ef2760a3d724a7b3da6952a'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Parkwoods - Donalda',
  'headerFullLocation': 'Parkwoods - Donalda, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 3,
  'suggestedBounds': {'ne': {'lat': 43.757758604500005,
    'lng': -79.32343823984928},
   'sw': {'lat': 43.7487585955, 'lng': -79.33587476015072}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8d9dcdd5fbbbb6b3003c7b',
       'name': 'Brookbanks Park',
       'location': {'address': 'Toronto',
        'lat': 43.751976046055574,
        'lng': -79.33214044722958,
        'labeledLatLngs': 

In [19]:
# define function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
# clean the json and structure it into a pandas dataframe
import json
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\Users\habib\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  import sys


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,Corrosion Service Company Limited,Construction & Landscaping,43.752432,-79.334661


In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

3 venues were returned by Foursquare.


Now create a function to repeat the process for all neighborhoods in North York

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID,CLIENT_SECRET,VERSION,lat,lng,radius,LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
ny_venues = getNearbyVenues(names=ny_data['Neighborhood'],
                                   latitudes=ny_data['Latitude'],
                                   longitudes=ny_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [24]:
# check how many venues were returned for each neighborhood
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",10,10,10,10,10,10
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",10,10,10,10,10,10
Don Mills,15,15,15,15,15,15
Downsview,17,17,17,17,17,17
"Fairview, Henry Farm, Oriole",10,10,10,10,10,10
Glencairn,5,5,5,5,5,5
Hillcrest Village,4,4,4,4,4,4
Humber Summit,2,2,2,2,2,2


In [25]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 73 uniques categories.


### Analyze Each Neighborhood in North York

In [26]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head()

,Neighborhood,Accessories Store,Airport,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,...,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [27]:
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,Accessories Store,Airport,Art Gallery,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,Baseball Field,...,Shopping Mall,Snack Place,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Tea Room,Thai Restaurant,Toy / Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.0,0.000000,0.00,0.100000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bayview Village,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.250000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bedford Park, Lawrence Manor East",0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.1,0.0,0.1,0.0,0.0,0.0
3,Don Mills,0.0,0.000000,0.066667,0.0,0.066667,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.066667,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Downsview,0.0,0.058824,0.000000,0.0,0.058824,0.00,0.058824,0.0,0.058824,...,0.058824,0.058824,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,"Fairview, Henry Farm, Oriole",0.0,0.000000,0.000000,0.0,0.000000,0.10,0.000000,0.0,0.000000,...,0.100000,0.000000,0.000000,0.0,0.0,0.1,0.0,0.1,0.0,0.0
6,Glencairn,0.0,0.000000,0.000000,0.2,0.000000,0.20,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Hillcrest Village,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Humber Summit,0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,"Humberlea, Emery",0.0,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.0,1.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0     Coffee Shop   0.2
1     Bridal Shop   0.1
2  Ice Cream Shop   0.1
3   Deli / Bodega   0.1
4            Bank   0.1


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3  Japanese Restaurant  0.25
4         Liquor Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0           Juice Bar   0.1
1  Italian Restaurant   0.1
2                Café   0.1
3                 Pub   0.1
4          Restaurant   0.1


----Don Mills----
                  venue  freq
0           Coffee Shop  0.07
1            Beer Store  0.07
2        Clothing Store  0.07
3            Restaurant  0.07
4  Caribbean Restaurant  0.07


----Downsview----
                  venue  freq
0         Grocery Store  0.18
1                  Park  0.12
2           Snack Place  0.06
3  Gym / Fitness Center  0.06
4               Ai

Put the information into a pandas dataframe

In [29]:
# define function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [30]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Fried Chicken Joint,Ice Cream Shop,Bridal Shop,Restaurant
1,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Food Truck
2,"Bedford Park, Lawrence Manor East",Indian Restaurant,Thai Restaurant,Juice Bar,Sushi Restaurant,Italian Restaurant
3,Don Mills,Restaurant,Discount Store,Caribbean Restaurant,Café,Japanese Restaurant
4,Downsview,Grocery Store,Park,Airport,Baseball Field,Gym / Fitness Center


### Cluster Neighborhoods

In [31]:
from sklearn.cluster import KMeans

Run KMeans to cluster the neighborhood into 5 clusters

In [32]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 3, 2])

Create dataframe that includes the cluster as well as the top 5 venues for each neighborhood

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ny_merged = ny_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Food & Drink Shop,Construction & Landscaping,Park,Women's Store,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Coffee Shop
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0.0,Accessories Store,Clothing Store,Vietnamese Restaurant,Boutique,Coffee Shop
3,M3B,North York,Don Mills,43.745906,-79.352188,0.0,Restaurant,Discount Store,Caribbean Restaurant,Café,Japanese Restaurant
4,M6B,North York,Glencairn,43.709577,-79.445073,0.0,Asian Restaurant,Bakery,Japanese Restaurant,Pub,Italian Restaurant


Visualize the resulting clusters

In [46]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[lat_ny, long_ny], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat,lon],
        radius=5,
        popup=label,
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the Clusters

Cluster 1

In [39]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North York,0.0,Intersection,Pizza Place,Hockey Arena,Portuguese Restaurant,Coffee Shop
2,North York,0.0,Accessories Store,Clothing Store,Vietnamese Restaurant,Boutique,Coffee Shop
3,North York,0.0,Restaurant,Discount Store,Caribbean Restaurant,Café,Japanese Restaurant
4,North York,0.0,Asian Restaurant,Bakery,Japanese Restaurant,Pub,Italian Restaurant
5,North York,0.0,Restaurant,Discount Store,Caribbean Restaurant,Café,Japanese Restaurant
6,North York,0.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Deli / Bodega
7,North York,0.0,Coffee Shop,Fried Chicken Joint,Ice Cream Shop,Bridal Shop,Restaurant
8,North York,0.0,Movie Theater,Electronics Store,Burger Joint,Restaurant,Shopping Mall
9,North York,0.0,Vietnamese Restaurant,Coffee Shop,Massage Studio,Bar,Caribbean Restaurant
10,North York,0.0,Café,Bank,Japanese Restaurant,Chinese Restaurant,Food Truck


Cluster 2

In [40]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,North York,1.0,Cafeteria,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop


Cluster 3

In [41]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
19,North York,2.0,Baseball Field,Caribbean Restaurant,Clothing Store,Coffee Shop,Comfort Food Restaurant


Cluster 4

In [42]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,North York,3.0,Furniture / Home Store,Pizza Place,Women's Store,Deli / Bodega,Chinese Restaurant


Cluster 5

In [43]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,4.0,Food & Drink Shop,Construction & Landscaping,Park,Women's Store,Deli / Bodega
14,North York,4.0,Basketball Court,Bakery,Construction & Landscaping,Park,Food Truck
22,North York,4.0,Park,Convenience Store,Women's Store,Chinese Restaurant,Clothing Store
